In [1]:
from jaxspec.ispec_synthetic_grid import *

In [8]:
#ranges = define_grid_ranges(tstep=2000, gstep=2, fstep=1.)
ranges = define_grid_ranges()

In [9]:
#precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "turbospectrum", number_of_processes=10)
#precompute_synthetic_grid(ranges, 1400, 1800, 120000, "ispecgrid_hband", "synthe", atmosphere_model='atlas', number_of_processes=10)

In [10]:
precompute_synthetic_grid(ranges, 500, 600, 80000, "ispecgrid_gaoes", "turbospectrum", number_of_processes=10)

# step wave (AA): 0.012500156252031047
# resolution: 439994.49999725807
# atmosphere model: marcs
[started] 3500 3.0 -1.0 0.03500    [started]1.78  0.03.00.0-1.0  0.00.4  [started]01.78
  0.03500 0.0  3.00.0  -0.750
 0.0 1.79 [started]0.0  0.0 35000.0  03.0
 [started]-0.75  3500 0.43.0  -0.5 1.790.0  1.8 0.00.0 [started]0.0   0.00.03500   00.03.0
  0-0.5[started]
 0.4  35001.8  3.00.0  0.0-0.25  0.00.0  01.81
 0.0 0.0 [started]0.0 0
 3500 [started]3.03.0  3500 -0.25  0.0 0.00.4

INFO:root:Spherical model atmosphere with radius 3.65e+11 cm


 1.82

[2023-11-10 13:40:31,992] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 3.65e+11 cm


  0.0 0.01.81 [started]0.0   0.003500
  3.00.0

...

[finished] 7500 3.5 -0.75 0.0 2.19 0.0 0.0 0.0 0
[started] 7500 3.5 0.5 0.0 2.25 0.0 0.0 0.0 0


INFO:root:Spherical model atmosphere with radius 2.05e+11 cm
[2023-11-10 15:01:06,712] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 2.05e+11 cm


[finished] 7500 3.5 -0.75 0.4 2.19 0.0 0.0 0.0 0
[started] 7500 3.5 0.5 0.4 2.25 0.0 0.0 0.0 0


INFO:root:Spherical model atmosphere with radius 2.05e+11 cm
[2023-11-10 15:01:10,157] [INFO] [turbospectrum:generate_spectrum:88]: Spherical model atmosphere with radius 2.05e+11 cm


[finished] 7500 3.5 -0.5 0.0 2.2 0.0 0.0 0.0 0
[started] 7500 4.0 -1.0 0.0 2.11 0.0 0.0 0.0 0
[finished] 7500 3.5 -0.5 0.4 2.2 0.0 0.0 0.0 0
[started] 7500 4.0 -1.0 0.4 2.11 0.0 0.0 0.0 0
[finished] 7500 3.5 -0.25 0.0 2.21 0.0 0.0 0.0 0
[started] 7500 4.0 -0.75 0.0 2.12 0.0 0.0 0.0 0
[finished] 7500 3.5 -0.25 0.4 2.21 0.0 0.0 0.0 0
[started] 7500 4.0 -0.75 0.4 2.12 0.0 0.0 0.0 0
[finished] 7500 3.5 0.0 0.0 2.22 0.0 0.0 0.0 0
[started] 7500 4.0 -0.5 0.0 2.13 0.0 0.0 0.0 0
[finished] 7500 3.5 0.0 0.4 2.22 0.0 0.0 0.0 0
[started] 7500 4.0 -0.5 0.4 2.13 0.0 0.0 0.0 0
[finished] 7500 4.0 -1.0 0.0 2.11 0.0 0.0 0.0 0
[started] 7500 4.0 -0.25 0.0 2.14 0.0 0.0 0.0 0
[finished] 7500 4.0 -1.0 0.4 2.11 0.0 0.0 0.0 0
[started] 7500 4.0 -0.25 0.4 2.14 0.0 0.0 0.0 0
[finished] 7500 3.5 0.25 0.0 2.23 0.0 0.0 0.0 0
[started] 7500 4.0 0.0 0.0 2.15 0.0 0.0 0.0 0
[finished] 7500 3.5 0.25 0.4 2.23 0.0 0.0 0.0 0
[started] 7500 4.0 0.0 0.4 2.15 0.0 0.0 0.0 0
[finished] 7500 3.5 0.5 0.0 2.25 0.0 0.0 0.0 0
[st